In [1]:
import cv2
import numpy as np

## Homography transform (image)

In [2]:
# get the 4 corner manually
def get_four_points(image):
    # Create a window to display the image
    cv2.namedWindow("Select Points")
    
    # Create an empty list to store the points
    points = []
    
    # Define the callback function for mouse events
    def mouse_callback(event, x, y, flags, param):
        # If the left mouse button is clicked, add the current point to the list
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            # Draw a circle at the current point
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
            # Update the display
            cv2.imshow("Select Points", image)

    # Display the image and wait for mouse clicks
    cv2.imshow("Select Points", image)
    cv2.setMouseCallback("Select Points", mouse_callback)
    cv2.waitKey(0)
    
    # Check if exactly 4 points were selected
    if len(points) != 4:
        print("Error: Please select exactly 4 points")
        return None
    
    #close the window
    cv2.destroyAllWindows()

    # Return the list of points
    return points
   


In [3]:
img= cv2.imread('board1.jpg')
src_pts= get_four_points(img)
height = 600
width= 700
# p to a numpy array
src_pts= np.array(src_pts)
dst_pts = np.array([[0, 0], [width, 0], [0, height ], [width, height ]], dtype="float32")
#homography transform
h, status = cv2.findHomography(src_pts, dst_pts)
img_out = cv2.warpPerspective(img, h, (width,height ))
cv2.imshow("result", img_out)
cv2.waitKey(0)
cv2.destroyAllWindows()
    


In [5]:
cv2.imshow("result", img_out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
def sort_coordinates(coord):
    detected_circles= [ [val[0], val[1]] for val in coord[0]]
    detected_circles= sorted(detected_circles, key=lambda x: x[1])
    coord2= [detected_circles[7*i:7*(i+1)] for i in range(0,6)]
    coord3= [sorted(val, key=lambda x: x[0]) for val in coord2]
    coord4= [val for sublist in coord3 for val in sublist]

    return coord4

In [11]:
img = img_out.copy()
def draw_circles_from_image(img):

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur the image
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    # Blur using 3 * 3 kernel.
    gray_blurred = cv2.blur(gray, (3, 3))
    
    # Apply Hough transform on the blurred image.
    detected_circles = cv2.HoughCircles(gray_blurred, 
                    cv2.HOUGH_GRADIENT, 1, 50, param1 = 95,
                param2 = 40, minRadius = 10, maxRadius = 55)
    
    # Draw circles that are detected.
    if detected_circles is not None:
    
        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(detected_circles))
        detected_circles = sort_coordinates(detected_circles)

        acc=1
        for pt in detected_circles:
            a, b = pt[0], pt[1]
    
            # Draw the circumference of the circle.
            cv2.circle(img, (a, b), 40, (0, 255, 0), 2)
    
            # Draw a small circle (of radius 1) to show the center.
            cv2.circle(img, (a, b), 1, (0, 0, 255), 3)
            if acc<11:
                cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            elif acc<21:
                cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            elif acc<31:
                cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 30), 2, cv2.LINE_AA)
            else:
                cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 60, 0), 2, cv2.LINE_AA)
            #draw a number in the center of the circle
            #cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            acc+=1
    return img

        
#show image
cv2.imshow('image',draw_circles_from_image(img))

cv2.waitKey(0)
cv2.destroyAllWindows()

In [144]:
detected_circles

In [7]:
board = [
    [".", ".", ".", "." , ".", ".",  "."], 
    [".", ".", ".", "." , ".", ".",  "."],
    [".", ".", ".", "." , ".", ".",  "."],
    [".", ".", ".", "." , ".", ".",  "."],
    [".", ".", ".", "." , ".", ".",  "."],
    [".", ".", ".", "." , ".", ".",  "."]
]

board= (np.array(board)).flatten()


In [10]:
import cv2
import numpy as np

# Load the image
img = img_out.copy()

# Define the coordinates and radius of each circle
detec
circles = np.uint16(np.around(detected_circles))



# Define the color range for red in HSV format
red_range_low = np.array([0,50,50])
red_range_high = np.array([10, 255, 255])

# Loop over each circle
num=1
red=0
r= 20
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
for x,y in circles:
    print("Circle #{}".format(num))
    print("Center: ({}, {})".format(x, y))
  # Extract the center pixel color in the HSV format
    circle_region = img[int(y-r):int(y+r), int(x-r):int(x+r)]
    if not circle_region.any():
        continue
    circle_hsv = cv2.cvtColor(circle_region, cv2.COLOR_BGR2HSV)
    hsv_pixel = np.mean(hsv_img[y-20: y+20 , x -20: x+20], axis=(0,1))
    #hsv_pixel= hsv_img[y-4,x] 
    #print(hsv_pixel, hsv_pixel2)
    # Check if the pixel color is within the red color range
    if hsv_pixel[0] >= red_range_low[0] and hsv_pixel[0] <= red_range_high[0] and hsv_pixel[1] >= red_range_low[1] and hsv_pixel[1] <= red_range_high[1] and hsv_pixel[2] >= red_range_low[2] and hsv_pixel[2] <= red_range_high[2]:
        print("Red")
        board[num-1]="X"
        red+=1
    else:
        print("Other")

    num+=1


print("nums red",red)

NameError: name 'circles' is not defined

In [25]:
r = 30
x= 544
y=552
lower_red = (0, 50, 50)
upper_red = (10, 255, 255)
circle_region = img[int(y-r):int(y+r), int(x-r):int(x+r)]
circle_hsv = cv2.cvtColor(circle_region, cv2.COLOR_BGR2HSV)
mask_red = cv2.inRange(circle_hsv, lower_red, upper_red)
mask_red

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
for i, square in enumerate(squares):
            row = i // 3
            col = i % 3
            x1, y1, x2, y2 = square
            # Add the square to the grid
            board[row, col] = 0

            # Crop the image to the bounding box of the square
            square_img = transformed_frame[y1:y2, x1:x2]

            # Convert the cropped image to grayscale
            square_gray = cv2.cvtColor(square_img, cv2.COLOR_BGR2GRAY)

            # Aplicar un filtro gaussiano para reducir el ruido
            blur = cv2.GaussianBlur(gray, (5, 5), 0)

            # Apply the Hough circle transform to detect circles
            circles = cv2.HoughCircles(square_gray, cv2.HOUGH_GRADIENT, dp=1, minDist=20, param1=50, param2=12, minRadius=20, maxRadius=30)


            if circles is not None:

                circles = circles[0]
                for (x, y, r) in circles:
                    # Extraer la región del círculo
                    circle_region = square_img[int(y-r):int(y+r), int(x-r):int(x+r)]
                    if not circle_region.any():
                        continue
                    circle_hsv = cv2.cvtColor(circle_region, cv2.COLOR_BGR2HSV)
                    
                    
                    # Definir rangos de color para rojo y verde
                    lower_red = (0, 50, 50)
                    upper_red = (10, 255, 255)
                    lower_green = (50, 50, 50)
                    upper_green = (70, 255, 255)
                    
                    # Segmentar el color del círculo utilizando los rangos de color definidos
                    mask_red = cv2.inRange(circle_hsv, lower_red, upper_red)
                    mask_green = cv2.inRange(circle_hsv, lower_green, upper_green)
                    
                    # Contar los píxeles de cada máscara para determinar el color predominante
                    count_red = cv2.countNonZero(mask_red)
                    count_green = cv2.countNonZero(mask_green)
                    
                    # Imprimir el resultado
                    if count_red > count_green:
                        board[row, col] = 1 
                                            
                    else:
                        board[row,col] = 2

In [15]:
print("Detected red circles")
board.reshape(6,7).tolist()

Detected red circles


[['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', 'X', '.'],
 ['.', '.', '.', '.', 'X', '.', '.']]

# Try with video

## Get circles 

In [5]:
def draw_circles_from_video(img):

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur the image
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    # Blur using 3 * 3 kernel.
    gray_blurred = cv2.blur(gray, (3, 3))
    
    # Apply Hough transform on the blurred image.
    detected_circles = cv2.HoughCircles(gray_blurred, 
                    cv2.HOUGH_GRADIENT, 1, 92, param1 = 82,
                param2 = 25, minRadius = 10, maxRadius = 55)
    
    # Draw circles that are detected.
    if detected_circles is not None:
    
        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(detected_circles))
        #detected_circles= [ [val[0], val[1]] for val in detected_circles[0]]
        detected_circles = sort_coordinates(detected_circles)

        acc=1
        for pt in detected_circles:
            a, b = pt[0], pt[1]
    
            # Draw the circumference of the circle.
            cv2.circle(img, (a, b), 40, (0, 255, 0), 2)
    
            # Draw a small circle (of radius 1) to show the center.
            cv2.circle(img, (a, b), 1, (0, 0, 255), 3)
            if acc<11:
                cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            elif acc<21:
                cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
            elif acc<31:
                cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 30), 2, cv2.LINE_AA)
            else:
                cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 60, 0), 2, cv2.LINE_AA)
            #draw a number in the center of the circle
            #cv2.putText(img, str(acc), (a, b), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            acc+=1
    return detected_circles,img

#show image
img=cv2.imread('board_lab.jpg')
cv2.imshow('image',draw_circles_from_video(img)[1])

cv2.waitKey(0)
cv2.destroyAllWindows()

## Circles to matrix

In [6]:
def circles2matrix(circles, img):
    # Define the color range for red in HSV format
    red_range_low = np.array([0,70,50])
    red_range_high = np.array([10, 255, 255])

    # Loop over each circle
    num=1
    red=0
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    for x,y in circles:
        print("Circle #{}".format(num))
        print("Center: ({}, {})".format(x, y))
    # Extract the center pixel color in the HSV format
        #circle_hsv = cv2.cvtColor(circle_region, cv2.COLOR_BGR2HSV)
        hsv_pixel = np.mean(hsv_img[y-20: y+20 , x -20: x+20], axis=(0,1))
        print("HSV color: {}".format(hsv_pixel))
        # Check if the pixel color is within the red color range
        if hsv_pixel[0] >= red_range_low[0] and hsv_pixel[0] <= red_range_high[0] and hsv_pixel[1] >= red_range_low[1] and hsv_pixel[1] <= red_range_high[1] and hsv_pixel[2] >= red_range_low[2] and hsv_pixel[2] <= red_range_high[2]:
            print("Red")
            board[num-1]="X"
            red+=1
        else:
            print("Other")

        num+=1

    print("Red circles: {}".format(red))

# do it for the image
#circles2matrix(detected_circles,img_out)

In [12]:
#open camera
cam = cv2.VideoCapture(0)
acc=0

while True:
    check, frame = cam.read()
    if acc == 0:
        img = frame.copy()
        src_pts= get_four_points(img)
        height = 600
        width= 700
        # p to a numpy array
        src_pts= np.array(src_pts)
        dst_pts = np.array([[0, 0], [width, 0], [0, height ], [width, height ]], dtype="float32")
        #homography transform
        
    h, status = cv2.findHomography(src_pts, dst_pts)
    img_out = cv2.warpPerspective(frame, h, (width,height ))

    detected_circles, _withCircles= draw_circles_from_video(img_out)
    cv2.imshow('video', frame)
    cv2.imshow("Hough Transform", img_out)
    cv2.imshow('With circles',_withCircles )
    acc+=1

    key = cv2.waitKey(1)
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()


Error: Please select exactly 4 points


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'findHomography'
> Overload resolution failed:
>  - srcPoints data type = 17 is not supported
>  - Expected Ptr<cv::UMat> for argument 'srcPoints'
>  - findHomography() missing required argument 'params' (pos 3)
>  - findHomography() missing required argument 'params' (pos 3)


In [79]:
cv2.imshow('image',img_out)
cv2.waitKey(0)
cv2.destroyAllWindows()